Use this utlity to update the returns and std_dev fields within investment-options.csv

In [1]:
%%javascript
IPython.OutputArea.prototype._should_scroll = function(lines) {
    return false;
}

<IPython.core.display.Javascript object>

In [2]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

import brownbear as bb

# Format price data.
pd.options.display.float_format = '{:0.2f}'.format

%matplotlib inline

In [3]:
# Set size of inline plots.
'''note: rcParams can't be in same cell as import matplotlib
   or %matplotlib inline
   
   %matplotlib notebook: will lead to interactive plots embedded within
   the notebook, you can zoom and resize the figure
   
   %matplotlib inline: only draw static images in the notebook
'''
plt.rcParams["figure.figsize"] = (10, 7)

Globals

In [4]:
# Set refresh_timeseries=True to download timeseries.  Otherwise /symbol-cache is used.
refresh_timeseries = True

In [5]:
# Read in dow30.csv
dow30 = pd.read_csv('dow30.csv')

# Remove the exchange from the beginning of the symbol.
def _symbol(row):
    return row['Symbol'].split(':')[-1].strip()

dow30['Symbol'] = dow30.apply(_symbol, axis=1)
dow30.drop(columns=['Exchange', 'Date added', 'Notes', 'Index weighting'], inplace=True)
dow30.rename(columns={'Symbol': 'Symbol',
                      'Company':'Description',
                      'Industry':'Asset Class'}, inplace=True)

dow30.set_index("Symbol", inplace=True)
dow30

,Description,Asset Class
Symbol,,
MMM,3M,Conglomerate
AXP,American Express,Financial services
AMGN,Amgen,Biopharmaceutical
AMZN,Amazon,Retailing
AAPL,Apple,Information technology
BA,Boeing,Aerospace and defense
CAT,Caterpillar,Construction and mining
CVX,Chevron,Petroleum industry
CSCO,Cisco,Information technology


In [6]:
# Read in gics-2-asset-class.csv
gics2asset_class = pd.read_csv('gics-2-asset-class.csv', skip_blank_lines=True, comment='#')
gics2asset_class.set_index("GICS", inplace=True)
gics2asset_class = gics2asset_class['Asset Class'].to_dict()
gics2asset_class

{'Conglomerate': 'US Stocks:Industrials',
 'Financial services': 'US Stocks:Financials',
 'Pharmaceutical industry': 'US Stocks:Healthcare',
 'Biopharmaceutical': 'US Stocks:Healthcare',
 'Information technology': 'US Stocks:Technology',
 'Aerospace and defense': 'US Stocks:Industrials',
 'Construction and mining': 'US Stocks:Industrials',
 'Petroleum industry': 'US Stocks:Energy',
 'Food industry': 'US Stocks:Consumer Staples',
 'Drink industry': 'US Stocks:Consumer Staples',
 'Chemical industry': 'US Stocks:Materials',
 'Retailing': 'US Stocks:Consumer Discretionary',
 'Apparel': 'US Stocks:Consumer Discretionary',
 'Fast-moving consumer goods': 'US Stocks:Consumer Staples',
 'Managed health care': 'US Stocks:Healthcare',
 'Telecommunication': 'US Stocks:Communication Services',
 'Broadcasting and entertainment': 'US Stocks:Communication Services',
 'Home Improvement': 'US Stocks:Consumer Discretionary',
 'Semiconductor industry': 'US Stocks:Technology',
 'Clothing industry': 'US Sto

In [7]:
# Map dow30 GICS sectors to brownbear defined asset classes.
def _asset_class(row):
    return gics2asset_class[row['Asset Class']]

dow30['Asset Class'] = dow30.apply(_asset_class, axis=1)

# Yahoo finance uses '-' where '.' is used in symbol names.
dow30.index = dow30.index.str.replace('.', '-', regex=False)
dow30

,Description,Asset Class
Symbol,,
MMM,3M,US Stocks:Industrials
AXP,American Express,US Stocks:Financials
AMGN,Amgen,US Stocks:Healthcare
AMZN,Amazon,US Stocks:Consumer Discretionary
AAPL,Apple,US Stocks:Technology
BA,Boeing,US Stocks:Industrials
CAT,Caterpillar,US Stocks:Industrials
CVX,Chevron,US Stocks:Energy
CSCO,Cisco,US Stocks:Technology


In [8]:
# Make symbols list.
symbols = list(dow30.index)
#symbols

In [9]:
# Get the timeseries for the symbols and compile into a single csv.
bb.fetch_timeseries(symbols, refresh=refresh_timeseries)
bb.compile_timeseries(symbols)

MMM AXP AMGN AMZN AAPL BA CAT CVX CSCO KO DIS GS HD HON IBM JNJ JPM MCD MRK MSFT NKE NVDA PG CRM SHW TRV UNH VZ V WMT 


In [10]:
# Read symbols timeseries into a dataframe.
df = pd.read_csv('symbols-timeseries.csv', skip_blank_lines=True, comment='#')
df.set_index("Date", inplace=True)
df

,MMM,AXP,AMGN,AMZN,AAPL,BA,CAT,CVX,CSCO,KO,...,NKE,NVDA,PG,CRM,SHW,TRV,UNH,VZ,V,WMT
Date,,,,,,,,,,,,,,,,,,,,,
2019-01-02,126.19,87.83,159.87,76.96,37.71,314.65,109.72,85.17,35.50,38.96,...,69.19,3.38,78.00,134.80,123.90,102.64,222.48,40.08,127.49,28.29
2019-01-03,121.44,86.11,157.44,75.01,33.95,302.10,105.49,83.53,34.21,38.72,...,67.96,3.17,77.45,129.68,120.16,100.79,216.41,40.22,122.89,28.15
2019-01-04,126.43,89.99,162.82,78.77,35.40,317.82,111.26,85.27,35.75,39.49,...,69.74,3.38,79.03,137.19,123.35,103.78,218.94,40.32,128.19,28.32
2019-01-07,126.14,90.48,165.01,81.48,35.32,318.82,111.32,86.37,35.99,38.97,...,70.74,3.56,78.72,141.43,125.09,103.45,219.36,40.58,130.50,28.66
2019-01-08,126.67,90.92,167.12,82.83,36.00,330.89,112.66,86.00,36.28,39.41,...,71.68,3.47,79.01,144.91,125.97,103.05,222.29,41.77,131.21,28.86
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2025-01-27,151.52,316.42,282.95,235.42,229.86,175.16,394.98,157.23,59.08,63.87,...,75.58,118.42,169.66,347.10,364.17,251.01,543.52,40.64,334.54,97.40
2025-01-28,151.40,317.04,280.30,238.15,238.26,177.78,390.29,156.21,59.43,62.36,...,74.39,128.99,166.19,359.95,364.17,248.43,542.48,40.40,334.48,97.29
2025-01-29,151.54,315.25,281.68,237.07,239.36,173.66,393.23,155.69,59.55,62.83,...,76.58,123.70,166.23,354.00,360.41,249.52,538.10,40.28,335.88,97.50


In [11]:
# Sample symbol.
symbol = 'MMM'

In [12]:
annual_returns = bb.annualized_returns(df, timeperiod='daily', years=1)
annual_returns[symbol]

np.float64(96.70059525394181)

In [13]:
# Calculate 1 month, 3 months, 1 year, 3 year, and 5 year annualized returns.
annual_returns_1mo = bb.annualized_returns(df, timeperiod='daily', years=1/12)
annual_returns_3mo = bb.annualized_returns(df, timeperiod='daily', years=3/12)
annual_returns_1yr = bb.annualized_returns(df, timeperiod='daily', years=1)
annual_returns_3yr = bb.annualized_returns(df, timeperiod='daily', years=3)
annual_returns_5yr = bb.annualized_returns(df, timeperiod='daily', years=5)

In [14]:
# Calculate 20 day annualized volatility.
daily_returns = df.pct_change()
years = bb.TRADING_DAYS_PER_MONTH / bb.TRADING_DAYS_PER_YEAR
vola = bb.annualized_standard_deviation(daily_returns, timeperiod='daily', years=years)
vola[symbol]

np.float64(0.22025003086965766)

In [15]:
# Calculate 20 day annualized downside volatility.
ds_vola = bb.annualized_standard_deviation(daily_returns, timeperiod='daily', years=years, downside=True)
ds_vola[symbol]

np.float64(0.09591841033447338)

In [16]:
# Resample df on a monthly basis.
df.index = pd.to_datetime(df.index)
monthly = df.resample('ME').ffill()
bb.print_full(monthly[symbol])

Date
2019-01-31   132.37
2019-02-28   138.00
2019-03-31   138.26
2019-04-30   126.10
2019-05-31   107.21
2019-06-30   116.33
2019-07-31   117.26
2019-08-31   109.53
2019-09-30   111.34
2019-10-31   111.74
2019-11-30   115.97
2019-12-31   120.52
2020-01-31   108.38
2020-02-29   102.87
2020-03-31    94.10
2020-04-30   104.72
2020-05-31   108.90
2020-06-30   108.59
2020-07-31   104.75
2020-08-31   114.52
2020-09-30   112.52
2020-10-31   112.37
2020-11-30   122.39
2020-12-31   123.85
2021-01-31   124.46
2021-02-28   125.06
2021-03-31   137.65
2021-04-30   140.84
2021-05-31   146.12
2021-06-30   142.94
2021-07-31   142.45
2021-08-31   141.21
2021-09-30   127.20
2021-10-31   129.57
2021-11-30   124.31
2021-12-31   129.86
2022-01-31   121.37
2022-02-28   109.72
2022-03-31   109.89
2022-04-30   106.45
2022-05-31   111.31
2022-06-30    96.48
2022-07-31   106.80
2022-08-31    93.66
2022-09-30    83.23
2022-10-31    94.75
2022-11-30    95.99
2022-12-31    91.38
2023-01-31    87.70
2023-02-28    8

In [17]:
# Calculate monthly returns.
monthly_returns = monthly.pct_change()
monthly_returns[symbol]

Date
2019-01-31     NaN
2019-02-28    0.04
2019-03-31    0.00
2019-04-30   -0.09
2019-05-31   -0.15
              ... 
2024-09-30    0.01
2024-10-31   -0.06
2024-11-30    0.04
2024-12-31   -0.03
2025-01-31    0.18
Freq: ME, Name: MMM, Length: 73, dtype: float64

In [18]:
# Calculate 3 year annualized standard deviation.
std_dev = bb.annualized_standard_deviation(monthly_returns, timeperiod='monthly', years=3)
std_dev[symbol]

np.float64(0.32480836822166714)

In [19]:
# Read investment-options-header.csv
lines = []
with open('investment-options-in.csv', 'r') as f:
    lines = [line.strip() for line in f]
lines

['# Description: DOW 30 investment options. 10/02/2020',
 '',
 '# Format',
 '"Investment Option","Description","Asset Class","1 mo","3 mo","1 Yr","3 Yr","5 Yr","Vola","DS Vola","Std Dev"',
 '# Note: "Description" field is optional']

In [20]:
# For each symbol, write out the 1 Yr, 3 Yr, 5 Yr, and std dev.
out = lines.copy()

for i, (index, row) in enumerate(dow30.iterrows()):

    symbol = index
    description = row['Description']
    asset_class = row['Asset Class']

    ret_1mo = annual_returns_1mo[symbol]
    ret_3mo = annual_returns_3mo[symbol]
    ret_1yr = annual_returns_1yr[symbol]
    ret_3yr = annual_returns_3yr[symbol]
    ret_5yr = annual_returns_5yr[symbol]
    
    if np.isnan(ret_3yr): ret_3yr = ret_1yr
    if np.isnan(ret_5yr): ret_5yr = ret_3yr

    _vola = vola[symbol]*100
    _ds_vola = ds_vola[symbol]*100
    sd = std_dev[symbol]*100

    out.append(
        '"{}","{}","{}","{:0.2f}","{:0.2f}","{:0.2f}","{:0.2f}","{:0.2f}","{:0.2f}","{:0.2f}","{:0.2f}"'
        .format(symbol, description, asset_class,
                ret_1mo, ret_3mo, ret_1yr, ret_3yr, ret_5yr, _vola, _ds_vola, sd)) 

In [21]:
# Write out asset-classes.csv
with open('investment-options.csv', 'w') as f:
    for line in out:
        f.write(line + '\n')